小麦数据集的下载方式：
```
wget https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt
```

In [1]:
import pandas as pd 
import xgboost as xgb 
import numpy as np 
data = pd.read_csv("data/seeds_dataset.txt",header=None,sep="\s+",converters={7:lambda x:int(x)-1})

data.rename(columns={7:"label"},inplace=True)

data.head(10)

,0,1,2,3,4,5,6,label
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,0
5,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,0
6,14.69,14.49,0.8799,5.563,3.259,3.586,5.219,0
7,14.11,14.10,0.8911,5.420,3.302,2.700,5.000,0
8,16.63,15.46,0.8747,6.053,3.465,2.040,5.877,0
9,16.44,15.25,0.8880,5.884,3.505,1.969,5.533,0


In [2]:
mask = np.random.rand(len(data)) < 0.8
train = data[mask]
test = data[~mask]

xgb_train = xgb.DMatrix(train.iloc[:,:6],label = train.label)
xgb_test = xgb.DMatrix(test.iloc[:,:6],label=test.label)

params = {
    'objective': 'multi:softmax',
    'eta': 0.1,
    'max_depth': 5,
    'num_class': 3
}

watchlist = [(xgb_train,'train'),(xgb_test,'test')]
num_round= 50
bst = xgb.train(params,xgb_train,num_round,watchlist)

[23:05:44] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-mlogloss:0.97370	test-mlogloss:0.99220
[1]	train-mlogloss:0.86848	test-mlogloss:0.90933
[2]	train-mlogloss:0.77856	test-mlogloss:0.83772
[3]	train-mlogloss:0.70058	test-mlogloss:0.77274
[4]	train-mlogloss:0.63218	test-mlogloss:0.71444
[5]	train-mlogloss:0.57235	test-mlogloss:0.66438
[6]	train-mlogloss:0.51972	test-mlogloss:0.62166
[7]	train-mlogloss:0.47377	test-mlogloss:0.58758
[8]	train-mlogloss:0.43245	test-mlogloss:0.55461
[9]	train-mlogloss:0.39554	test-mlogloss:0.52758
[10]	train-mlogloss:0.36257	test-mlogloss:0.50333
[11]	train-mlogloss:0.33191	test-mlogloss:0.47824
[12]	train-mlogloss:0.30457	test-mlogloss:0.45627
[13]	train-mlogloss:0.28068	test-mlogloss:0.43823
[14]	train-mlogloss:0.25848	test-mlogloss:0.42292
[15]	tra

In [3]:
pred = bst.predict(xgb_test)
error_rate = np.sum(pred!=test.label)/test.shape[0]
print('测试集错误率（softmax）：{}'.format(error_rate))

测试集错误率（softmax）：0.1


In [4]:
params['objective'] = 'multi:softprob'
bst = xgb.train(params,xgb_train,num_round,watchlist)
pred_prob = bst.predict(xgb_test)
pred_label = np.argmax(pred_prob,axis=1)
error_rate = np.sum(pred_label!=test.label)/test.shape[0]
print('测试集错误率（softprob）：{}'.format(error_rate))

[23:05:44] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-mlogloss:0.97370	test-mlogloss:0.99220
[1]	train-mlogloss:0.86848	test-mlogloss:0.90933
[2]	train-mlogloss:0.77856	test-mlogloss:0.83772
[3]	train-mlogloss:0.70058	test-mlogloss:0.77274
[4]	train-mlogloss:0.63218	test-mlogloss:0.71444
[5]	train-mlogloss:0.57235	test-mlogloss:0.66438
[6]	train-mlogloss:0.51972	test-mlogloss:0.62166
[7]	train-mlogloss:0.47377	test-mlogloss:0.58758
[8]	train-mlogloss:0.43245	test-mlogloss:0.55461
[9]	train-mlogloss:0.39554	test-mlogloss:0.52758
[10]	train-mlogloss:0.36257	test-mlogloss:0.50333
[11]	train-mlogloss:0.33191	test-mlogloss:0.47824
[12]	train-mlogloss:0.30457	test-mlogloss:0.45627
[13]	train-mlogloss:0.28068	test-mlogloss:0.43823
[14]	train-mlogloss:0.25848	test-mlogloss:0.42292
[15]	tr